# Training a Model to Predict US Congress Votes

## 1.: Creating an aggregate DataFrame

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
attendance_df = pd.read_csv('./data/attendance_2024.csv')
senators_df = pd.read_csv('./data/senators_2024.csv')
vote_details_df = pd.read_csv('./data/vote_details_2024.csv')

In [ ]:
mapping = {
    'Bill Passed': 1,
    'Nomination Confirmed': 1,
    'Cloture Motion Agreed to': 1,
    'Motion to Proceed Rejected': 0,
    'Amendment Rejected': 0,
    'Cloture Motion Rejected': 0,
    'Cloture on the Motion to Proceed Rejected': 0,
    'Motion Agreed to': 1,
    'Point of Order Well Taken': 1,
    'Joint Resolution Passed': 1,
    'Joint Resolution Defeated': 0,
    'Motion to Table Failed': 0,
    'Motion to Proceed Agreed to': 1,
    'Cloture on the Motion to Proceed Agreed to': 1,
    'Bill Defeated': 0,
    'Motion to Adjourn Agreed to': 1,
    'Motion to Adjourn Rejected': 0,
    'Motion Rejected': 0,
    'Motion to Table Agreed to': 1,
    'Motion to Refer Rejected': 0,
    'Motion for Attendance Agreed to': 1,
    'Motion to Discharge Rejected': 0,
    'Veto Sustained': 0,
    'Amendment Agreed to': 1,
    'Motion to Reconsider Agreed to': 1
}

vote_details_df['passed'] = vote_details_df['result'].map(mapping)
vote_details_df.head()

In [14]:
aggregated_df = vote_details_df["vote_number"]

parties = senators_df['party'].unique()

parties

array(['D', 'R', 'I'], dtype=object)